# Summary

We need to rerun `add_adjacency_distances.ipynb` code for the following domains:

***training_dataset***

~72% of rows finished.

```raw
[('database_id=G3DSA%3A3.40.50.1440', 62807),
 ('database_id=G3DSA%3A3.20.80.10', 43126),
 ('database_id=G3DSA%3A3.30.1050.10', 34935),
 ('database_id=G3DSA%3A3.30.110.60', 11123),
 ('database_id=G3DSA%3A1.10.10.440', 12939),
 ('database_id=G3DSA%3A1.10.10.41', 2895),
 ('database_id=G3DSA%3A3.30.1140.32', 17973),
 ('database_id=G3DSA%3A3.30.1120.40', 3065),
 ('database_id=G3DSA%3A2.60.40.1200', 53),
 ('database_id=G3DSA%3A1.10.10.250', 18310),
 ('database_id=G3DSA%3A2.60.40.1090', 45878),
 ('database_id=G3DSA%3A3.50.7.10', 57672),
 ('database_id=G3DSA%3A3.50.70.10', 6224),
 ('database_id=G3DSA%3A1.10.10.10', 3400746),
 ('database_id=G3DSA%3A3.20.20.70', 1701341),
 ('database_id=G3DSA%3A1.25.40.10', 2105462),
 ('database_id=G3DSA%3A3.20.90.10', 5021),
 ('database_id=G3DSA%3A3.40.640.10', 940256),
 ('database_id=G3DSA%3A1.10.10.190', 11),
 ('database_id=G3DSA%3A1.10.10.410', 28270),
 ('database_id=G3DSA%3A1.10.10.180', 1311),
 ('database_id=G3DSA%3A3.30.1040.10', 11),
 ('database_id=G3DSA%3A2.60.40.1240', 16791),
 ('database_id=G3DSA%3A1.10.10.500', 1171),
 ('database_id=G3DSA%3A3.30.110.40', 21055),
 ('database_id=G3DSA%3A3.50.50.60', 1600839),
 ('database_id=G3DSA%3A1.10.10.390', 428),
 ('database_id=G3DSA%3A2.60.40.10', 2604549),
 ('database_id=G3DSA%3A3.30.1120.30', 4708),
 ('database_id=G3DSA%3A1.10.10.400', 18468),
 ('database_id=G3DSA%3A2.60.40.1120', 184571),
 ('database_id=G3DSA%3A1.10.10.430', 1060)]
```

***validation_dataset***

~82% of rows finished.

```raw
[('database_id=G3DSA%3A2.102.10.10', 2531),
 ('database_id=G3DSA%3A2.20.140.10', 182),
 ('database_id=G3DSA%3A1.25.40.270', 340),
 ('database_id=G3DSA%3A2.10.25.30', 23),
 ('database_id=G3DSA%3A2.30.29.30', 30432),
 ('database_id=G3DSA%3A1.25.40.20', 40925),
 ('database_id=G3DSA%3A2.120.10.90', 1824),
 ('database_id=G3DSA%3A2.30.170.40', 1510),
 ('database_id=G3DSA%3A2.100.10.20', 45),
 ('database_id=G3DSA%3A2.20.90.10', 4),
 ('database_id=G3DSA%3A2.30.130.10', 2377),
 ('database_id=G3DSA%3A1.20.59.10', 964),
 ('database_id=G3DSA%3A2.170.14.10', 59),
 ('database_id=G3DSA%3A2.20.170.10', 10),
 ('database_id=G3DSA%3A2.100.10.30', 349),
 ('database_id=G3DSA%3A2.10.290.10', 180),
 ('database_id=G3DSA%3A1.25.10.30', 970),
 ('database_id=G3DSA%3A2.30.18.10', 189)]
```

***test_dataset***

~95.6% of rows finished.

```raw
[('database_id=G3DSA%3A2.40.290.10', 610),
 ('database_id=G3DSA%3A2.30.30.390', 56),
 ('database_id=G3DSA%3A2.40.200.10', 14),
 ('database_id=G3DSA%3A2.40.230.10', 589),
 ('database_id=G3DSA%3A2.40.230.20', 399),
 ('database_id=G3DSA%3A2.40.270.10', 8268),
 ('database_id=G3DSA%3A2.40.180.10', 5573)]
```

# Imports

In [1]:
import concurrent.futures
import concurrent.futures.process
import importlib
import os
import shlex
import shutil
import subprocess
import sys
import warnings
from functools import partial
from itertools import islice
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psutil
import pyarrow as pa
import pyarrow.parquet as pq
import yaml

from kmbio import PDB
from kmtools import structure_tools

In [2]:
%matplotlib inline

pd.set_option("max_columns", 100)

In [3]:
SRC_PATH = Path.cwd().joinpath('..', 'src').resolve(strict=True)

if SRC_PATH.as_posix() not in sys.path:
    sys.path.insert(0, SRC_PATH.as_posix())

import helper
importlib.reload(helper)

<module 'helper' from '/gpfs/fs0/scratch/p/pmkim/strokach/datapkg/adjacency-net-v2/src/helper/__init__.py'>

In [ ]:
%run spark.ipynb

# Parameters

In [4]:
NOTEBOOK_PATH = Path(os.getenv("CI_JOB_NAME", "add_adjacency_distances_retry"))
NOTEBOOK_PATH

PosixPath('add_adjacency_distances_retry')

In [5]:
OUTPUT_PATH = Path(os.getenv('OUTPUT_DIR', NOTEBOOK_PATH.name)).resolve()
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH

PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/datapkg/adjacency-net-v2/notebooks/add_adjacency_distances_retry')

In [6]:
Path.cwd().expanduser()

PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/datapkg/adjacency-net-v2/notebooks')

In [7]:
DEBUG = "CI" not in os.environ    

if DEBUG:
    TASK_ID = 1
    TASK_COUNT = 1
    ADJACENCY_MATRIX_PARQUET_PATH = (
        Path(os.getenv("DATAPKG_OUTPUT_DIR"))
        .joinpath("adjacency-net-v2", "master", "training_dataset", "adjacency_matrix.parquet")
    )
else:
    assert TASK_ID is not None
    assert TASK_COUNT is not None
    assert ADJACENCY_MATRIX_PARQUET_PATH is not None

assert ADJACENCY_MATRIX_PARQUET_PATH.is_dir()

TASK_ID, TASK_COUNT, ADJACENCY_MATRIX_PARQUET_PATH

(1,
 1,
 PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/databin/adjacency-net-v2/master/training_dataset/adjacency_matrix.parquet'))

In [8]:
ADJACENCY_MATRIX_PARQUET_PATH = os.getenv("ADJACENCY_MATRIX_PARQUET_PATH")

TASK_ID = int(TASK_ID) if TASK_ID is not None else None
TASK_COUNT = int(TASK_COUNT) if TASK_COUNT is not None else None
ADJACENCY_MATRIX_PARQUET_PATH = (
    Path(ADJACENCY_MATRIX_PARQUET_PATH).expanduser()
    if ADJACENCY_MATRIX_PARQUET_PATH is not None
    else None
)

TASK_ID, TASK_COUNT, ADJACENCY_MATRIX_PARQUET_PATH

(1, 1, None)

In [9]:
if DEBUG:
    %load_ext autoreload
    %autoreload 2

In [10]:
FAILED_DOMAIN_NAMES = {
    'training_dataset': [t[0] for t in 
        [('database_id=G3DSA%3A3.40.50.1440', 62807),
         ('database_id=G3DSA%3A3.20.80.10', 43126),
         ('database_id=G3DSA%3A3.30.1050.10', 34935),
         ('database_id=G3DSA%3A3.30.110.60', 11123),
         ('database_id=G3DSA%3A1.10.10.440', 12939),
         ('database_id=G3DSA%3A1.10.10.41', 2895),
         ('database_id=G3DSA%3A3.30.1140.32', 17973),
         ('database_id=G3DSA%3A3.30.1120.40', 3065),
         ('database_id=G3DSA%3A2.60.40.1200', 53),
         ('database_id=G3DSA%3A1.10.10.250', 18310),
         ('database_id=G3DSA%3A2.60.40.1090', 45878),
         ('database_id=G3DSA%3A3.50.7.10', 57672),
         ('database_id=G3DSA%3A3.50.70.10', 6224),
         ('database_id=G3DSA%3A1.10.10.10', 3400746),
         ('database_id=G3DSA%3A3.20.20.70', 1701341),
         ('database_id=G3DSA%3A1.25.40.10', 2105462),
         ('database_id=G3DSA%3A3.20.90.10', 5021),
         ('database_id=G3DSA%3A3.40.640.10', 940256),
         ('database_id=G3DSA%3A1.10.10.190', 11),
         ('database_id=G3DSA%3A1.10.10.410', 28270),
         ('database_id=G3DSA%3A1.10.10.180', 1311),
         ('database_id=G3DSA%3A3.30.1040.10', 11),
         ('database_id=G3DSA%3A2.60.40.1240', 16791),
         ('database_id=G3DSA%3A1.10.10.500', 1171),
         ('database_id=G3DSA%3A3.30.110.40', 21055),
         ('database_id=G3DSA%3A3.50.50.60', 1600839),
         ('database_id=G3DSA%3A1.10.10.390', 428),
         ('database_id=G3DSA%3A2.60.40.10', 2604549),
         ('database_id=G3DSA%3A3.30.1120.30', 4708),
         ('database_id=G3DSA%3A1.10.10.400', 18468),
         ('database_id=G3DSA%3A2.60.40.1120', 184571),
         ('database_id=G3DSA%3A1.10.10.430', 1060)]
    ],
    'validation_dataset': [t[0] for t in 
        [('database_id=G3DSA%3A2.102.10.10', 2531),
         ('database_id=G3DSA%3A2.20.140.10', 182),
         ('database_id=G3DSA%3A1.25.40.270', 340),
         ('database_id=G3DSA%3A2.10.25.30', 23),
         ('database_id=G3DSA%3A2.30.29.30', 30432),
         ('database_id=G3DSA%3A1.25.40.20', 40925),
         ('database_id=G3DSA%3A2.120.10.90', 1824),
         ('database_id=G3DSA%3A2.30.170.40', 1510),
         ('database_id=G3DSA%3A2.100.10.20', 45),
         ('database_id=G3DSA%3A2.20.90.10', 4),
         ('database_id=G3DSA%3A2.30.130.10', 2377),
         ('database_id=G3DSA%3A1.20.59.10', 964),
         ('database_id=G3DSA%3A2.170.14.10', 59),
         ('database_id=G3DSA%3A2.20.170.10', 10),
         ('database_id=G3DSA%3A2.100.10.30', 349),
         ('database_id=G3DSA%3A2.10.290.10', 180),
         ('database_id=G3DSA%3A1.25.10.30', 970),
         ('database_id=G3DSA%3A2.30.18.10', 189)]
    ],
    'test_dataset': [t[0] for t in     
        [('database_id=G3DSA%3A2.40.290.10', 610),
         ('database_id=G3DSA%3A2.30.30.390', 56),
         ('database_id=G3DSA%3A2.40.200.10', 14),
         ('database_id=G3DSA%3A2.40.230.10', 589),
         ('database_id=G3DSA%3A2.40.230.20', 399),
         ('database_id=G3DSA%3A2.40.270.10', 8268),
         ('database_id=G3DSA%3A2.40.180.10', 5573)]
    ],
}

# `DATAPKG`

In [11]:
DATAPKG = {}

In [12]:
DATAPKG['pdb-ffindex'] = {
    'pdb_mmcif_ffindex': (
        Path(os.environ['DATAPKG_OUTPUT_DIR'])
        .joinpath("pdb-ffindex", "master", "pdb_mmcif_ffindex", "pdb-mmcif")
    ),
}

# Load data

In [13]:
Path(os.getenv("DATAPKG_OUTPUT_DIR"))

PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/databin')

In [14]:
files = sorted(sum(
    (list(
        Path(os.getenv("DATAPKG_OUTPUT_DIR"))
        .joinpath("adjacency-net-v2", "master", dataset_name, "adjacency_matrix.parquet", domain_name)
        .glob("*.parquet")
    )
    for dataset_name in ["training_dataset", "validation_dataset", "test_dataset"]
    for domain_name in FAILED_DOMAIN_NAMES[dataset_name])
, []))

In [15]:
{f.parent.parent for f in files}

{PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/databin/adjacency-net-v2/master/test_dataset/adjacency_matrix.parquet'),
 PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/databin/adjacency-net-v2/master/training_dataset/adjacency_matrix.parquet'),
 PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/databin/adjacency-net-v2/master/validation_dataset/adjacency_matrix.parquet')}

In [16]:
df = (
    pq.ParquetFile(files[0])
    .read_row_group(0, use_pandas_metadata=True)
    .to_pandas(integer_object_nulls=True)
    .set_index("__index_level_0__")
)

In [17]:
df.head(1)

,uniparc_id,sequence,database,interpro_name,interpro_id,domain_start,domain_end,domain_length,structure_id,model_id,chain_id,pc_identity,alignment_length,mismatches,gap_opens,q_start,q_end,s_start,s_end,evalue_log10,bitscore,qseq,sseq,a2b,b2a,residue_idx_1,residue_idx_2,residue_id_1,residue_id_2,residue_aa_1,residue_aa_2,residue_idx_1_corrected,residue_idx_2_corrected
__index_level_0__,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
287082306,UPI0005B61E35,APWYHVVMEDDDGQPVHTYLAEAQLRSEMRDEHPEQPSMDELARTI...,Gene3D,Hemimethylated DNA-binding domain superfamily,IPR036623,1,57,57,1VBV,0.0,A,87.76,49.0,6.0,0.0,1.0,49.0,29.0,77.0,-25.522879,94.7,APWYHVVMEDDDGQPVHTYLAEAQLRSEMRDEHPEQPSMDELARTIRKQ,APWYHVVMEDDNGLPVHTYLAEAQLSSELQDEHPEQPSMDELAQTIRKQ,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, ...","[28.0, 28.0, 28.0, 29.0, 29.0, 29.0, 29.0, 29....","[29.0, 30.0, 49.0, 28.0, 30.0, 31.0, 47.0, 48....","[49.0, 49.0, 49.0, 50.0, 50.0, 50.0, 50.0, 50....","[50.0, 51.0, 70.0, 49.0, 51.0, 52.0, 68.0, 69....","[A, A, A, P, P, P, P, P, P, W, W, W, W, W, W, ...","[P, W, E, A, W, Y, L, A, E, A, P, Y, H, Y, L, ...","[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 2.0, 21.0, 0.0, 2.0, 3.0, 19.0, 20.0, 21..."


# Run pipeline

In [18]:
def worker(data):
    row = helper.to_namedtuple(data)
    results = {}
    try:
        results['residue_idx_1'], results['residue_idx_2'], results['distances'] = (
            helper.get_adjacency_with_distances(
                row, max_cutoff=12, min_cutoff=None, structure_url_prefix=STRUCTURE_URL_PREFIX
            )
        )
#         results['residue_idx_1'] = results['residue_idx_1'].tolist()
#         results['residue_idx_2'] = results['residue_idx_2'].tolist()
#         results['distances'] = results['distances'].tolist()
        results["error"] = None
    except Exception as e:
        results = {
            "residue_idx_1": None,
            "residue_idx_2": None,
            "distances": None,
            "error": f"{type(e)}: {e}",
        }
    return results

## Run worker for single row

In [19]:
start = 0
row = next(islice(df.itertuples(), start, start + 1))

In [20]:
STRUCTURE_URL_PREFIX = f"ff://{DATAPKG['pdb-ffindex']['pdb_mmcif_ffindex']}?"
STRUCTURE_URL_PREFIX

'ff:///gpfs/fs0/scratch/p/pmkim/strokach/databin/pdb-ffindex/master/pdb_mmcif_ffindex/pdb-mmcif?'

In [21]:
# residue_idx_1, residue_idx_2, distance = helper.get_adjacency_with_distances(
#     row, max_cutoff=12, min_cutoff=None, structure_url_prefix=STRUCTURE_URL_PREFIX
# )

In [22]:
worker(row._asdict())

{'residue_idx_1': None,
 'residue_idx_2': None,
 'distances': None,
 'error': "<class 'FileNotFoundError'>: File not found: '1vbv.cif.gz'"}

## Run for all rows

In [44]:
def get_new_file(file, suffix):
    file_parts = list(file.parts)
    file_parts[-4] = file_parts[-4] + suffix
    new_file = Path(*file_parts)
    return new_file

get_new_file(files[0], "_wdistances_failed")

PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/databin/adjacency-net-v2/master/test_dataset_wdistances_failed/adjacency_matrix.parquet/database_id=G3DSA%3A2.30.30.390/part-00000-68abfd61-d8ca-427e-ae54-7bdd8a9779b1-c000.snappy.parquet')

### Examine subset of files

In [50]:
print(len(files))

42


In [51]:
if DEBUG:
    files_arrow_not_implemented = set()

    for file in files:
        try:
            ds = pq.ParquetFile(file)
            table = ds.read_row_group(0, use_pandas_metadata=True)
        except pa.ArrowNotImplementedError:
            files_arrow_not_implemented.add(file)

    files = [f for f in files if f not in files_arrow_not_implemented]

In [52]:
print(len(files))

42


In [85]:
if DEBUG:
    files_to_run = files[:10]
else:
    files_to_run = files

In [86]:
print(len(files_to_run))

10


In [ ]:
for file in files_to_run:
    ds = pq.ParquetFile(file)
    new_file = get_new_file(file, "_wdistances")
    new_file.parent.mkdir(parents=True, exist_ok=True)
    writer_failed_is_initialized = False
    using_spark_sql = False
    for row_group in range(ds.num_row_groups):
        # Read row group (or entire file using Spark SQL if PyArrow fails)
        try:
            df = (
                ds.read_row_group(row_group, use_pandas_metadata=True)
                .to_pandas(integer_object_nulls=True)
                .set_index("__index_level_0__")
            )
        except pa.ArrowNotImplementedError:
            using_spark_sql = True
            df = spark.read.parquet(file.as_posix()).toPandas()

        try:
            with concurrent.futures.ProcessPoolExecutor(psutil.cpu_count(logical=False)) as pool:
                futures = pool.map(
                    worker,
                    (t._asdict() for t in df[helper.GET_ADJACENCY_WITH_DISTANCES_ROW_ATTRIBUTES].itertuples()),
                    chunksize=1)
                results = list(futures)
                results_df = pd.DataFrame(results)
                df["residue_idx_1_corrected"] = results_df["residue_idx_1"].values
                df["residue_idx_2_corrected"] = results_df["residue_idx_2"].values
                df["distances"] = results_df["distances"].values
                df["error_adding_distances"] = results_df["error"].values
                num_errors = df["error_adding_distances"].notnull().sum()
                if num_errors:
                    print(f"Encountered {num_errors} errors when parsing file '{file}'.")
        except concurrent.futures.process.BrokenProcessPool as e:
            warnings.warn(
                f"ProcessPool crashed while processing row_group '{row_group}' in file '{file}'."
                f"The error is '{type(e)}': {e}."
            )
            break

        df_succeeded = df[
            df[["residue_idx_1_corrected", "residue_idx_2_corrected", "distances"]].notnull().all(axis=1)
        ]
        df_failed =  df[
            df[["residue_idx_1_corrected", "residue_idx_2_corrected", "distances"]].isnull().all(axis=1)
        ]
        assert len(df_succeeded) + len(df_failed) == len(df)

        # Write successful results
        table = pa.Table.from_pandas(df_succeeded, preserve_index=True)
        if row_group == 0:
            writer = pq.ParquetWriter(new_file, table.schema, version="2.0", flavor="spark")
        writer.write_table(table)

        # Write failed results
        if not df_failed.empty:
            table_failed = pa.Table.from_pandas(df_failed, preserve_index=True)
            if not writer_failed_is_initialized:
                new_file_failed = get_new_file(file, "_wdistances_failed")
                new_file_failed.parent.mkdir(parents=True, exist_ok=True)
                writer_failed = pq.ParquetWriter(new_file_failed, table_failed.schema, version="2.0", flavor="spark")
                writer_failed_is_initialized = True
            writer_failed.write_table(table_failed)

        # print(len(df_succeeded), len(df_failed))

        if using_spark_sql:
            break

    writer.close()
    if writer_failed_is_initialized:
        writer_failed.close()

Encountered 56 errors when parsing file '/gpfs/fs0/scratch/p/pmkim/strokach/databin/adjacency-net-v2/master/test_dataset/adjacency_matrix.parquet/database_id=G3DSA%3A2.30.30.390/part-00000-68abfd61-d8ca-427e-ae54-7bdd8a9779b1-c000.snappy.parquet'.
0 56
14 0
589 0
399 0
8268 0
610 0
1311 0
11 0


In [80]:
file

PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/databin/adjacency-net-v2/master/test_dataset/adjacency_matrix.parquet/database_id=G3DSA%3A2.30.30.390/part-00000-68abfd61-d8ca-427e-ae54-7bdd8a9779b1-c000.snappy.parquet')

In [81]:
new_file

PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/databin/adjacency-net-v2/master/test_dataset_wdistances/adjacency_matrix.parquet/database_id=G3DSA%3A2.30.30.390/part-00000-68abfd61-d8ca-427e-ae54-7bdd8a9779b1-c000.snappy.parquet')

In [82]:
new_file_failed

PosixPath('/gpfs/fs0/scratch/p/pmkim/strokach/databin/adjacency-net-v2/master/test_dataset_wdistances_failed/adjacency_matrix.parquet/database_id=G3DSA%3A2.30.30.390/part-00000-68abfd61-d8ca-427e-ae54-7bdd8a9779b1-c000.snappy.parquet')

In [ ]:
if DEBUG:
    display(df.head())

    # Make sure that the file we wrote makes sense
    for file in files_to_run:
        new_file = get_new_file(file)

        ds = pq.ParquetFile(file)
        ds_new = pq.ParquetFile(new_file)
        assert ds.num_row_groups == ds_new.num_row_groups

        for row_group in range(ds.num_row_groups):
            df = (
                ds.read_row_group(0, use_pandas_metadata=True)
                .to_pandas(integer_object_nulls=True)
                .set_index("__index_level_0__")
            )
            df_new = (
                ds_new.read_row_group(0, use_pandas_metadata=True)
                .to_pandas(integer_object_nulls=True)
#                 .set_index("__index_level_0__")
            )
            shared_columns = [
                c for c in df.columns
                if c in df_new.columns
                and c not in [
                    'a2b', 'b2a', 'residue_idx_1', 'residue_idx_2',
                    'residue_id_1', 'residue_id_2', 'residue_aa_1', 'residue_aa_2',
                    "residue_idx_1_corrected", "residue_idx_2_corrected"]
            ]
            assert (df[shared_columns] == df_new[shared_columns]).all().all()
            assert all(
                (l1 == l2).all() 
                for l1, l2
                in zip(df["residue_id_1"].values, df_new["residue_id_1"].values)
            )
            assert all(
                (l1.shape != l2.shape or not (l1 == l2).all())
                for l1, l2
                in zip(df["residue_idx_1_corrected"].values, df_new["residue_idx_1_corrected"].values)
            )